In [14]:
import pandas as pd
import pymannkendall as mk
import matplotlib.pyplot as plt

In [15]:
well_data = pd.read_excel('../data/Data-Wells.xlsx')
rm_nd = well_data.loc[well_data['1,4-Dioxane Results (ppb)'] != 'nd'].copy()
rm_capital_nd = rm_nd.loc[rm_nd['1,4-Dioxane Results (ppb)'] != 'ND'].copy()
rm_capital_nd['1,4-Dioxane Results (ppb)'] = rm_capital_nd['1,4-Dioxane Results (ppb)'].map(lambda x: x.replace(',', ''))
rm_capital_nd['1,4-Dioxane Results (ppb)'] = rm_capital_nd['1,4-Dioxane Results (ppb)'].map(lambda x: x.replace('<', ''))
res = rm_capital_nd
res.loc[:, 'ds']= pd.to_datetime(res['Date Sampled'])
res.loc[:, 'dioxane_results']= pd.to_numeric(res['1,4-Dioxane Results (ppb)'])
di_time = res.iloc[:, lambda df: df.columns.str.contains('Well Name|ds|dioxane_results',
                                              case=False)].copy()
di_time.rename(columns = {'dioxane_results':'y'}, inplace = True)
di_time.head(5)

,Well Name,ds,y
16,110 Parkland Plaza,1986-08-22,17.0
17,110 Parkland Plaza,1986-09-19,17.0
18,110 Parkland Plaza,1986-10-10,12.0
19,110 Parkland Plaza,1986-10-10,1.0
20,110 Parkland Plaza,1987-11-09,7.0


In [17]:
grouped = di_time.groupby('Well Name')
mk_res = []
for name, group in grouped:

    try:
        data = group.y
        trend, h, p, z, Tau, s, var_s, slope, intercept = mk.original_test(data)

        mk_res.append([name, trend, h, p, z, Tau, s, var_s, slope, intercept])
    except ZeroDivisionError:
        # print(group)
        print("zero division error", name)
        # plt.plot(group.ds, group.y)
        # plt.show()

    except KeyboardInterrupt:
        break
    except:
        print("[mk error]", name)
        continue


mk_df = pd.DataFrame(mk_res, columns=['Well name', 'trend', 'h', 'p', 'z', 'Tau', 's', 'var_s','slope' , 'intercept'])
mk_df


zero division error 2655 Dexter Rd
zero division error 2828 Dexter Rd
zero division error 3041 Dexter Rd
zero division error 3225 Dexter Rd
zero division error 3250 Kingwood
zero division error 3432 Ferry St
zero division error 3537 Elizabeth Rd
zero division error 3550 Elizabeth Rd
zero division error 371 Parkland Plaza #1
zero division error 4061-4065 Jackson Rd
zero division error 4530 Jackson Rd
zero division error 460 Rose
zero division error 4709 Breezewood Ct
zero division error 777 S. Wagner Road
zero division error HCMW-2d
zero division error Honey Creek/W Huron Dr
zero division error INF-UV1-A
zero division error IW-1
zero division error LS-1
zero division error LS-2
zero division error LS-3
zero division error LS-5
zero division error MW-120s
zero division error MW-122d
zero division error MW-123s
zero division error MW-124d
zero division error MW-127d
zero division error MW-127s
zero division error MW-128s
zero division error MW-131d
zero division error MW-135
zero division

,Well name,trend,h,p,z,Tau,s,var_s,slope,intercept
0,110 Parkland Plaza,decreasing,True,2.326843e-10,-6.338055,-0.591195,-846.0,17774.666667,-0.200000,12.300000
1,170 Aprill,no trend,False,7.427813e-01,-0.328172,-0.039683,-25.0,5348.333333,0.000000,10.500000
2,175 Jackson Plaza,increasing,True,1.278999e-11,6.770996,0.616541,984.0,21076.666667,19.795139,-127.263889
3,2575 Valley,increasing,True,2.772549e-04,3.635674,0.594737,113.0,949.000000,4.844444,22.477778
4,2601 Dexter Rd,increasing,True,7.469328e-03,2.675161,0.722222,26.0,87.333333,1.000000,-2.000000
...,...,...,...,...,...,...,...,...,...,...
302,Tributary/Outfall,no trend,False,1.000000e+00,0.000000,0.333333,1.0,3.666667,1.650000,-0.050000
303,Tributary/Park Rd,no trend,False,8.641073e-02,1.714643,0.800000,8.0,16.666667,0.512500,2.375000
304,Varsity Ford,no trend,False,2.415666e-01,-1.171080,-0.333333,-5.0,11.666667,0.000000,3.000000
305,Weber's Drain 1st Sister Lake,no trend,False,1.000000e+00,0.000000,-1.000000,-1.0,1.000000,-19.000000,249.000000
